In [1]:
from __future__ import print_function
import tensorflow as tf
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
# Using local folder MNIST_data after reading it the first time
# mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W, b):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
def max_pool_2x2(x):
    # max_pool_2x2 downsamples a feature map by 2X
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

# First convolutional layer - maps one grayscale image to 32 feature maps.
with tf.name_scope('conv11'):
    W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1))
    b_conv1 = bias_variable([32])
    h_conv1 = conv2d(x_image, W_conv1, b_conv1)
#     h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# Pooling layer - downsamples by 2X.
with tf.name_scope('pooling1'):
    h_pool1 = max_pool_2x2(h_conv1)

# Second convolutional layer -- maps 32 feature maps to 64.
with tf.name_scope('conv12'):
#     W_conv2 = weight_variable([5, 5, 32, 64])
    W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1))
#     W_conv2 = tf.Variable(tf.zeros([5, 5, 32, 64]), name='Weights')
    b_conv2 = bias_variable([64])
    h_conv2 = conv2d(h_pool1, W_conv2, b_conv2)
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# Second pooling layer.
with tf.name_scope('pooling2'):
    h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
with tf.name_scope('fc1'):
#     W_fc1 = weight_variable([7 * 7 * 64, 1024])
#     W_fc1 = tf.Variable(tf.zeros([7 * 7 * 64, 1024]), name='Weights')
    W_fc1 = tf.Variable(tf.truncated_normal([7*7*64,1024], stddev=0.1), name="w1")
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.add(tf.matmul(h_pool2_flat, W_fc1), b_fc1)
#     h_fc1 = tf.nn.softmax(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1 = tf.nn.relu(h_fc1)

# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Map the 1024 features to 10 classes, one for each digit
with tf.name_scope('fc2'):
#     W_fc2 = weight_variable([1024, 10])
    W_fc2 = tf.Variable(tf.truncated_normal([1024,10], stddev=0.1), name="w2")
#     W_fc2 = tf.Variable(tf.zeros([1024, 10]), name='Weights')
    b_fc2 = bias_variable([10])
#     pred = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    pred_logits = tf.add(tf.matmul(h_fc1_drop, W_fc2), b_fc2)
    pred = tf.nn.softmax(pred_logits)
#     pred = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
#     pred = tf.nn.softmax(tf.matmul(x, W) + b)
# with tf.name_scope('Model'):
# # Model
#     pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
# Minimize error using cross entropy
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred_logits))
#     cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
#     cost = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=y, logits=pred))
with tf.name_scope('AdamOptimizer'):
# Gradient Descent
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
# Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()
# Start training
with tf.Session() as sess:
# Run the initializer
    sess.run(init)
# op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter('tmp/tensorflow_logs/cnn/train', graph=tf.get_default_graph())
    summary_writer_test = tf.summary.FileWriter('tmp/tensorflow_logs/cnn/test')
    total_batch = int(mnist.train.num_examples/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        for i in range(total_batch):
            step = epoch * total_batch + i
            if step%10==0:
                test_x, test_y = mnist.test.next_batch(batch_size)
                # Get Test Summary for one batch and add summary to TensorBoard
                summary = sess.run(merged_summary_op, 
                                   feed_dict={x: test_x, y: test_y, keep_prob: 1.0})
                summary_writer_test.add_summary(summary, step)
            else:
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                               keep_prob: 0.75})
                # Get Train Summary for one batch and add summary to TensorBoard, and train
                c, summary = sess.run([cost, merged_summary_op],
                                      feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
                summary_writer.add_summary(summary, step)
                # Compute average loss
                avg_cost += c / total_batch
        # Display logs per epoch step
#         if (epoch+1) % display_epoch == 0:
        print("Epoch:", '%d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
#         print(sess.run(pred)[0])
    print("Optimization Finished!")
    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))
    print("Run the command line:\n" \
    "--> tensorboard --logdir=/tmp/tensorflow_logs " \
    "\nThen open http://0.0.0.0:6006/ into your web browser")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 1 cost= 0.658663583
Epoch: 2 cost= 0.081308805
Epoch: 3 cost= 0.062135621
Epoch: 4 cost= 0.056739183
Epoch:

In [2]:
tf.reset_default_graph()